In [1]:
import pandas
import datetime
import math
import os
import numpy
from scipy import signal, ndimage, stats
from scipy.optimize import curve_fit
from davitpy.models import *
from davitpy import utils
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
import seaborn as sns
%matplotlib inline

In [2]:
# Some constants
equTrghCutoffMLat = 40.
polTrghCutoffMLat = 70.
# We'll only look at NA continent, So we 
# choose the corresponding MLONS - 260, 20
mlonNAList = range( -180, 181, 2 )#range( -100, 21, 2 )
# Some cutoffs to verify goodness of fit
cutoffKSPval = 0.7
cutoffKSDstat = 0.25
cutOffTrghMinUpper = 65.
cutOffTrghMinLower = 45.
cutOffPrcntErrorFit = 0.1 # 10%
cutOffLatCnt = 20

In [3]:
def convert_to_datetime(row):
    currDateStr = str( int( row["dateStr"] ) )
#     return currDateStr
    if row["timeStr"] < 10:
        currTimeStr = "000" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 100:
        currTimeStr = "00" + str( int( row["timeStr"] ) )
    elif row["timeStr"] < 1000:
        currTimeStr = "0" + str( int( row["timeStr"] ) )
    else:
        currTimeStr = str( int( row["timeStr"] ) )
    return datetime.datetime.strptime( currDateStr\
                    + ":" + currTimeStr, "%Y%m%d:%H%M" )

selDT = datetime.datetime( 2011, 4, 9, 9, 0 )
anlyzCoords = 'mag'

In [4]:
inpColList = [ "dateStr", "timeStr", "Mlat",\
              "Mlon", "med_tec", "dlat", "dlon" ]
inpFile = "../data/tec-medFilt-20110409.txt"
medFiltTECDF = pandas.read_csv(inpFile, delim_whitespace=True,\
                                    header=None, names=inpColList)
medFiltTECDF["date"] = medFiltTECDF.apply( convert_to_datetime, axis=1 )
medFiltTECDF.head()

,dateStr,timeStr,Mlat,Mlon,med_tec,dlat,dlon,date
0,20110409,800,11.0,34.0,4.9,1.0,2.0,2011-04-09 08:00:00
1,20110409,800,11.0,36.0,7.2,1.0,2.0,2011-04-09 08:00:00
2,20110409,800,11.0,38.0,7.3,1.0,2.0,2011-04-09 08:00:00
3,20110409,800,11.0,64.0,19.0,1.0,2.0,2011-04-09 08:00:00
4,20110409,800,11.0,66.0,19.3,1.0,2.0,2011-04-09 08:00:00


In [5]:
# Analyze the selected time
selTecDF = medFiltTECDF[ medFiltTECDF["date"] == selDT ].reset_index(drop=True)
selTecDF.head()

,dateStr,timeStr,Mlat,Mlon,med_tec,dlat,dlon,date
0,20110409,900,11.0,32.0,10.8,1.0,2.0,2011-04-09 09:00:00
1,20110409,900,11.0,34.0,11.9,1.0,2.0,2011-04-09 09:00:00
2,20110409,900,11.0,70.0,33.0,1.0,2.0,2011-04-09 09:00:00
3,20110409,900,11.0,72.0,34.0,1.0,2.0,2011-04-09 09:00:00
4,20110409,900,11.0,74.0,34.8,1.0,2.0,2011-04-09 09:00:00


In [6]:
def gauss_function(x, a, x0, sigma):
    return a*numpy.exp(-(x-x0)**2/(2*sigma**2))

In [7]:
BndMlonArr = []
BndEquMlatArr = []
BndPolMlatArr = []
minTecMlatArr = []
minTecValArr = []
BndEquTecValArr = []
BndPolTecValArr = []
currTimeArr = []
selMlonList = [ 300., 320., 20., 80. ]
for ind, sMlon in enumerate(mlonNAList):
    if sMlon < 0.:
        sMlon = sMlon + 360.
    selMlonDF = selTecDF[ (selTecDF["Mlon"] == sMlon) &\
                          (selTecDF["Mlat"] >= equTrghCutoffMLat) &\
                          (selTecDF["Mlat"] <= polTrghCutoffMLat)]
    # If no significant number of values are found discard
    if len( selMlonDF["med_tec"].values ) < 5.:
        continue
    tecGaussFitArr = numpy.max( selMlonDF["med_tec"].values ) - selMlonDF["med_tec"].values
    mlatPltArr = numpy.arange(equTrghCutoffMLat-20, polTrghCutoffMLat+20)
    try:
        popt, pcov = curve_fit(gauss_function, selMlonDF["Mlat"].values,\
                               tecGaussFitArr, p0 = [2, 52., 1.])
    except:
        print "failed fit at MLON-->", sMlon
        continue
    fwhmEqu = popt[1] - popt[2]*2.355/2.
    fwhmPol = popt[1] + popt[2]*2.355/2.
    tecValFwhmEqu = gauss_function(fwhmEqu, *popt)
    tecValFwhmPol = gauss_function(fwhmPol, *popt)
    # Test goodness of fit
    ksTestTecArr = numpy.array( [ gauss_function(l, *popt)\
                                for l in selMlonDF["Mlat"].values.tolist() ] )
    ksDStat, ksPVal = stats.ks_2samp( tecGaussFitArr, ksTestTecArr )
    # we setup a few conditions to discard bad fits
    # 1) location of trough min should be between 45 nad 65 MLAT
    if ((popt[1] > cutOffTrghMinUpper)\
            | (popt[1] < cutOffTrghMinLower)):
        continue
    # 2) if percent error in any of fit parameters is more 
    # than 10 % (cutoff) then skip
    if ( ( pcov[0,0]**0.5/popt[0] > cutOffPrcntErrorFit  ) \
        | ( pcov[1,1]**0.5/popt[1] > cutOffPrcntErrorFit  ) \
        | ( pcov[2,2]**0.5/popt[2] > cutOffPrcntErrorFit  ) ):
        continue
    # 3) Number of latitudes should be greater than 20.
    if len( selMlonDF["Mlat"].values ) < cutOffLatCnt:
        continue
    # 4) If p-val from the KS TEST is low discard
    # or if KS Stat is high discard
    if ksPVal < cutoffKSPval:
        continue
    if ksDStat > cutoffKSDstat:
        continue
    # Now we have good fits. Get appropriate boundary locations
    # and TEC values.
    # GET Trough min loc and tec val
    minTrghLoc = min(list(selMlonDF["Mlat"].values), key=lambda x:abs(x-popt[1]))
    minTrghTecVal = selMlonDF[ selMlonDF["Mlat"] == minTrghLoc ]["med_tec"].values[0]
    # GET Trough equ loc and tec val
    equTrghLoc = min(list(selMlonDF["Mlat"].values), key=lambda x:abs(x-fwhmEqu))
    equTrghTecVal = selMlonDF[ selMlonDF["Mlat"] == equTrghLoc ]["med_tec"].values[0]
    # GET Trough pol loc and tec val
    polTrghLoc = min(list(selMlonDF["Mlat"].values), key=lambda x:abs(x-fwhmPol))
    polTrghTecVal = selMlonDF[ selMlonDF["Mlat"] == polTrghLoc ]["med_tec"].values[0]
    # Append the values to arrays
    BndMlonArr.append( sMlon )
    BndEquMlatArr.append( equTrghLoc )
    BndPolMlatArr.append( polTrghLoc )
    minTecMlatArr.append( minTrghLoc )
    minTecValArr.append( minTrghTecVal )
    BndEquTecValArr.append( equTrghTecVal )
    BndPolTecValArr.append( polTrghTecVal )
    currTimeArr.append( selDT )
# Store data in a DF
trghBndDF = pandas.DataFrame({
            "BndMlon" : BndMlonArr,
            "BndEquMlat" : BndEquMlatArr,
            "BndPolMlat" : BndPolMlatArr,
            "minTecMlat" : minTecMlatArr,
            "minTecVal" : minTecValArr,
            "BndEquTecVal" : BndEquTecValArr,
            "BndPolTecVal" : BndPolTecValArr,
            "date" : currTimeArr
            })
trghBndDF.head()

failed fit at MLON--> 206.0
failed fit at MLON--> 222.0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/dist-packages/scipy/optimize/minpack.py:779: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


failed fit at MLON--> 32
failed fit at MLON--> 46
failed fit at MLON--> 48
failed fit at MLON--> 50
failed fit at MLON--> 54
failed fit at MLON--> 56
failed fit at MLON--> 58
failed fit at MLON--> 62
failed fit at MLON--> 70
failed fit at MLON--> 74
failed fit at MLON--> 76
failed fit at MLON--> 116
failed fit at MLON--> 128
failed fit at MLON--> 170
failed fit at MLON--> 172


,BndEquMlat,BndEquTecVal,BndMlon,BndPolMlat,BndPolTecVal,date,minTecMlat,minTecVal
0,55.0,8.4,244.0,68.0,7.7,2011-04-09 09:00:00,61.0,2.5
1,55.0,7.8,246.0,67.0,7.7,2011-04-09 09:00:00,61.0,2.6
2,54.0,8.5,248.0,67.0,6.7,2011-04-09 09:00:00,60.0,2.7
3,54.0,8.3,250.0,67.0,6.5,2011-04-09 09:00:00,60.0,3.1
4,54.0,8.0,252.0,66.0,6.8,2011-04-09 09:00:00,60.0,3.8


In [ ]:
print numpy.sort(trghBndDF["BndMlon"].values)

[   0.    2.    4.    6.   12.   16.   18.   88.   90.  244.  246.  248.
  250.  252.  254.  256.  258.  260.  262.  264.  268.  270.  272.  274.
  276.  296.  298.  300.  302.  312.  314.  316.  318.  320.  330.  332.
  338.  340.  342.  344.  346.  348.  350.  352.  354.  356.  358.]


In [ ]:
# set colorbar
seaMap = ListedColormap(sns.color_palette("Spectral_r"))
# flatui = ["#3182bd", "#9ecae1", "#7fcdbb", "#ffeda0", "#feb24c", "#f03b20"]
# sns.set_palette(flatui)
# seaMap = ListedColormap( sns.color_palette() )
# Plot map
f = plt.figure(figsize=(12, 8))
ax = f.add_subplot(1,1,1)
m1 = utils.plotUtils.mapObj(boundinglat=30., gridLabels=True, coords=anlyzCoords, ax=ax, datetime=selDT)
xVec, yVec = m1(list(selTecDF["Mlon"]), list(selTecDF["Mlat"]), coords=anlyzCoords)
# BND data
xVecEquBnd, yVecEquBnd = m1(list(trghBndDF["BndMlon"]), list(trghBndDF["BndEquMlat"]), coords="mag")
xVecPolBnd, yVecPolBnd = m1(list(trghBndDF["BndMlon"]), list(trghBndDF["BndPolMlat"]), coords="mag")
xVecMinTrghBnd, yVecMinTrghBnd = m1(list(trghBndDF["BndMlon"]), list(trghBndDF["minTecMlat"]), coords="mag")
tecPlot = m1.scatter( xVec, yVec , c=selTecDF["med_tec"], s=40.,\
           cmap=seaMap, alpha=0.7, zorder=5., \
                     edgecolor='none', marker="s", vmin=0., vmax=20. )

eqPlot = m1.scatter( xVecEquBnd, yVecEquBnd , s=10.,\
                     c='k', marker="^", zorder=7. )
poPlot = m1.scatter( xVecPolBnd, yVecPolBnd , s=10.,\
                     c='k', marker="v", zorder=7. )
mtPlot = m1.scatter( xVecMinTrghBnd, yVecMinTrghBnd , s=15.,\
                     c='r', marker="*", zorder=7. )

cbar = plt.colorbar(tecPlot, orientation='vertical')
cbar.set_label('TEC', size=15)

/usr/local/lib/python2.7/dist-packages/mpl_toolkits/basemap/__init__.py:1767: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  axisbgc = ax.get_axis_bgcolor()
